In [55]:
import PyPDF2
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
pdfreader = PyPDF2.PdfReader('budget_speech.pdf')

In [13]:
data = ''
for page in pdfreader.pages:
    data = data +"\n\n"+page.extract_text()

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(data)

In [38]:
documents = [Document(page_content=chunk) for chunk in chunks]

In [44]:
db = Chroma.from_documents(
        documents = documents,
        embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"),
        persist_directory="./chroma_db"  # Directory to save the database
    )
db.persist()

/var/folders/74/0gd30npx2mj_nbjyj17fr5m00000gn/T/ipykernel_1346/2831834621.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [56]:
llm = ChatGroq(model = 'llama-3.1-8b-instant')
parser = StrOutputParser()

In [54]:
template = '''I will provide you question and the document which is having answer of that question inside it. 
Give me answer of question in 2 lines. Mention "Dont Know Answer" if you cant find answer in document.
Question:{question}
Document:{document}'''
prompt = PromptTemplate(template=template, input_variables=['question','document'])

In [58]:
chain = prompt | llm | parser

In [92]:
question = 'What is budget for AI?'
similar_docs = db.similarity_search(question)
data = '\n\n'.join([d.page_content for d in similar_docs])
answer = chain.invoke({'question':question,'document':data})
print(answer)

I couldn't find the answer to your question "What is budget for AI" in the document. 
Dont Know Answer


In [96]:
print(similar_docs[3])

page_content='19  
 
Budget Estimates 2025 -26 
112. Coming to 2025 -26, the total receipts other than  borrowings  and the 
total expenditure are estimated at ` 34.96  lakh crore and ` 50.65  lakh crore 
respectively. The net tax receipts are estimated at ` 28.37  lakh crore.  
113. The fiscal  deficit  is estimated  to be 4.4 per cent  of GDP . 
114. To finance the fiscal deficit, the net market borrowings from dated 
securities are estimated at ` 11.54 lakh crore. The balance financing is expected 
to come from small savings and other sources. The gross market borrowings 
are estimated at ` 14.8 2 lakh crore.  
I will now move  to Part  B.'
